In [35]:
# Metadata so that you know what I'm working with:
!python --version
!mypy --version

Python 3.7.0 (default, Oct  9 2018, 10:31:47) 
[GCC 7.3.0]
mypy 0.800


# MyPy Notes

Python 3 introduces the use of type annotation sytax with [PEP526](https://www.python.org/dev/peps/pep-0526/). I've written this notebook with nb_mypy.

## Why bother:
- Catch common errors.
    - Would have saved me pain with horrible Bytes vs String Errors on Python3 upgrade.
- Still optional.

> note: Much of this functionality is still evolving - there are some workarounds for Pythons 3.6 - 3.8 that are not required at 3.9. 3.5 has different syntax again.

# Basics

Trad Python is _dynamically typed_:

In [36]:
def hi(name):
    return f'Hello {name}'

You can insert any data type, whether or not it makes sense:

In [37]:
for name in ['Tim', 42, ['Tim', 'Sibling', 'Nephew'], None, {'Really': 'no'}, b'Tim']:
    print(hi(name))

Hello Tim
Hello 42
Hello ['Tim', 'Sibling', 'Nephew']
Hello None
Hello {'Really': 'no'}
Hello b'Tim'


Using mypy:


In [38]:
%%writefile delme.py
def hi(name: str) -> str:
    return f'Hello {name}'

hi('Tim')
hi(432)
hi(['Tim'])

Writing delme.py


In [39]:
!mypy delme1.py

delme1.py:5: error: Argument 1 to "hi" has incompatible type "int"; expected "str"
delme1.py:6: error: Argument 1 to "hi" has incompatible type "List[str]"; expected "str"
Found 2 errors in 1 file (checked 1 source file)


MyPy also allows declaring variable types:

In [78]:
%%writefile delme.py
foo: str
foo = 42

bar: bool = False
baz: bool = 'False'   # This sort of thing causes nasty gotchas in some Rose code 😥

Overwriting delme.py


In [79]:
!mypy delme.py

delme.py:2: error: Incompatible types in assignment (expression has type "int", variable has type "str")
delme.py:5: error: Incompatible types in assignment (expression has type "str", variable has type "bool")
Found 2 errors in 1 file (checked 1 source file)


# Other simple examples:
You can add type hints to all sorts of function signatures:

In [40]:
%%writefile delme.py
def p():
    print('Hello World')
    
a = p()
print(f'a = {a}')

def q() -> None:
    print('Yo!')
    
b = q()

Overwriting delme.py


In [41]:
!mypy delme.py

delme.py:10: error: "q" does not return a value
Found 1 error in 1 file (checked 1 source file)


# More complex examples:

That example was really trivial, right? I mean it was an utterly deterministic function with one possible type of input and output...

What if we want:
- a dictionary of {str: float}?
- a list of integers (or floats, or strings, or whatever)?

> note: This is a thing which changes at Python 3.9 so that you could just do `list[str]`. Because I'm mostly using 3.7 this example uses the `typing` module to provide this functionality.



In [47]:
%%writefile delme.py
from typing import List

def greet_group(names: List[str]) -> None:
    for name in names:
        print(f'Bore Da, {name}')
    
greet_group(['Tom', 'Dick', 'Harry', 12.2])

Overwriting delme.py


In [48]:
!mypy delme.py

delme.py:7: error: List item 3 has incompatible type "float"; expected "str"
Found 1 error in 1 file (checked 1 source file)


To be more flexible you could use `collections.abc.Iterable` to allow any iterable type to be passed into your function:

In [67]:
%%writefile delme.py
from collections.abc import Iterable

def greet_ppl(names: Iterable[str]) -> None:
    for name in names:
        print(name)
        
greet_ppl(('Alice', 'Bob', 'Charlie the spy'))
greet_ppl(['Tim', 'Sibling', 'Nepphen'])
greet_ppl({'Me': 'Tim', 'Sibling': 'Simon', 'Nephew': 'Matthew'}.values())

Overwriting delme.py


In [68]:
!mypy delme.py

Success: no issues found in 1 source file


### For cases when the input can have multiple types use `Union` or `Optional`:

> `Union` == `Optional`

In [70]:
%%writefile delme.py
from typing import Union, Optional

def greet(name: Optional[str]=None) -> str:
    if name is None:
        name = 'World'
    return f'Hello {name}'

greet()
greet('Everyone')

Overwriting delme.py


In [71]:
!mypy delme.py

Success: no issues found in 1 source file


# Cheatsheet!

MyPy has a nice [cheatsheet](https://mypy.readthedocs.io/en/stable/cheat_sheet_py3.html)

# Debugging:

In [95]:
%%writefile delme.py
from typing import Union, Dict

foo: str = "Any old text"
reveal_type(foo)

bar: Union[str, None, int] = 54
reveal_type(bar)

baz: Dict[str, Dict[str, int]] = {'Tim': {'Favourite Number': 42}}
reveal_type(baz)

def 

Overwriting delme.py


In [96]:
!mypy delme.py

delme.py:12: error: invalid syntax
Found 1 error in 1 file (errors prevented further checking)


In [ ]:
# Write an e